<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 100px">

# Capstone Project: Classifying Logistics Research Papers
## Part 2 : Add Label 

---

 [Part 1: Get Text](01.Get_Text.ipynb) | **Part 2: Add Label** | [Part 3: EDA](03.EDA.ipynb) | [Part 4: Gridsearch Classification](04.Gridsearch_Classification.ipynb) | [Part 5: Neural Network Classification](05.NeuralNet_Classification.ipynb) | [Part 6: Model Evaluation](06.Model_Evaluation.ipynb) | [Part 7: Final Model](07.Final_Model.ipynb) 

---

### Introducion


This notebook focuses on labeling the topic category from abstracts by calling the Typhoon LLM API. It feeds the abstracts of files into the API to retrieve the corresponding answers from Typhoon LLM.

### Import Library

In [1]:
import pandas as pd
import numpy as np
import time
import os
from dotenv import load_dotenv
from openai import OpenAI

### Call Typhoon API

In [75]:
# Check get key
success = load_dotenv()
print(f"Load successful: {success}")

Load successful: True


In [76]:
# Check call API
client = OpenAI(
   api_key= os.getenv('API_KEY'),
   base_url='https://api.opentyphoon.ai/v1'
)

chat_completion = client.chat.completions.create(
    model="typhoon-v1.5x-70b-instruct",
    messages=[{"role": "user", "content": "สวัสดี"}]
)
print(chat_completion.choices[0].message.content)

สวัสดีค่ะ! ยินดีต้อนรับเข้าสู่การแชทของเรา หากคุณมีคำถามหรือต้องการความช่วยเหลือใดๆ กรุณาแจ้งให้เราทราบ เรายินดีที่จะช่วยคุณ.


### Labeling the Topic Category

In [78]:
# Function call Typhoon API
def get_llm_values(text, system_message):
    try:
        chat_completion = client.chat.completions.create(
            model="typhoon-v1.5x-70b-instruct",
            messages=[
                {"role": "system", "content": system_message},
                {"role": "user", "content": text}
            ]
        )
        category = chat_completion.choices[0].message.content.strip()
        time.sleep(8)  # Avoid rate limit (10 requests/min)
        
        return category
        
    except Exception as e:
        print(f"Error categorizing text: {e}")
        return None

In [79]:
# Define topic category message
topic_category_message = (
    "Your task is to ranking these categories from given text into.\n"
    "categories = {'Procurement', 'Manufacturing/Production', 'Inventory Management', "
    "'Logistics and Distribution', 'Demand Planning and Forecasting', 'Import-Export and International Trade', 'Customer Service' }\n "
    "If the text does not relate to any category on this list, answer 'Others' \n"
    "Answer with only category name"
)

In [479]:
df = pd.read_csv('../data/reseacrh_text.csv')
df.head()

,project,abstract,content
0,การจัดทำคู่มือขั้นตอนการดำเนินการการนำเข้าคราฟ...,ผู้วิจัยได้ตระหนักถึงความยุ่งยากของขั้นตอนการน...,การจัดทำคู่มือขั้นตอนการดำเนินการการนำเข้าคราฟ...
1,การเสนอแนวทางในการพัฒนาและสร้างความสัมพันธ์กับ...,งานวิจัยครั้งนี้มีวัตถุประสงค์เพื่อเสนอแนวทางใ...,การเสนอแนวทางในการพัฒนาและสร้างความสัมพันธ์กับ...
2,การพัฒนามาตรฐานรถขนส่งวัตถุอันตรายที่เข้ามาในค...,ดำเนินธุรกิจเป็นผู้นำเข้า และจัดจำหน่ายสินค้าก...,การพัฒนามาตรฐานรถขนส่งวัตถุอันตรายที่เข้ามาในค...
3,แนวทางการปรับปรุงกระบวนการการส่งเอกสารใบกำกับภ...,การวิจัยครั้งนี้มีวัตถุประสงค์ เพื่อศึกษาขั้นต...,แนวทางการปรับปรุงกระบวนการการส่งเอกสารใบกำกับภ...
4,การศึกษาเทคนิคการพยากรณ์ยอดขายสายไฟที่เหมาะสม,จากสถานการณ์การแพร่ระบาดของเชื้อไวรัสโคโรนา 20...,การศึกษาเทคนิคการพยากรณ์ยอดขายสายไฟที่เหมาะสม ...


In [81]:
# Add Label for Topic Category
start_time = time.time()

df['category'] = df['abstract'].apply(lambda x: get_llm_values(x, topic_category_message))

end_time = time.time()

# Calculate elapsed time in minutes
runtime_minutes = (end_time - start_time) / 60
print(f"Run time for label category: {runtime_minutes:.0f} minutes")

Run time for label category: 53 minutes


In [48]:
# Get category id start by 0 (easy to use for model)
category_mapping = {
    'Procurement': 0,
    'Manufacturing/Production': 1,
    'Inventory Management' : 2,
    'Logistics and Distribution': 3,
    'Demand Planning and Forecasting': 4,
    'Import-Export and International Trade': 5,
    'Customer Service' : 6,
    'Others': 7
}

# Add a new 'topic_category_id' column based on the mapping
df['category_id'] = check['category'].map(category_mapping)

In [47]:
# Check Category Distribution
df['category'].value_counts()

category
Inventory Management                     102
Logistics and Distribution                71
Import-Export and International Trade     41
Procurement                               30
Manufacturing/Production                  30
Others                                    28
Demand Planning and Forecasting           23
Customer Service                          15
Name: count, dtype: int64

In [51]:
# Save to output CSV
df.to_csv('../data/reseacrh_label_added.csv', index=False)

---

### Futures Improvement

Label Multi-Categories and Keywords to Improve Model Accuracy or Other Objectives

In [15]:
# # Define message to extract keywords
# keywords_message = (
#     "Your task is extract keywords that contain in given abstract \n"
#     "The total number of keywords should be between 3 to 5. \n"
#     "Keywords can be in either English or Thai. \n"
#     "If the text appears inside parentheses, it should not be considered as a keyword. \n"
#     "Return the keywords as a concatenated string, separated by commas."
# )

# # Define message to extract multiple-category from paper
# multi_topic_category_message = (
#     "Your task is to rank the correlation of the given text with the following categories\n"
#     "categories = {'Procurement', 'Manufacturing/Production', 'Inventory Management', "
#     "'Logistics and Distribution', 'Demand Planning and Forecasting', 'Import-Export and International Trade', "
#     " 'Customer Service', 'Others' }\n"
#     "Respond with a dictionary like {1: 'Inventory Management', 2: 'Logistics and Distribution', \n"
#     "3: 'Demand Planning', and so on for all 8 categories.")



In [2]:
# # Add Ranks of Topic Category
# start_time = time.time()

# df['multi_category'] = df['abstract'].apply(lambda x: get_llm_values(x, multi_topic_category_message))

# end_time = time.time()

# # Calculate elapsed time in minutes
# runtime_minutes = (end_time - start_time) / 60
# print(f"Run time for label mullti-category: {runtime_minutes:.0f} minutes")

# # Add keywords
# start_time = time.time()

# df['keywords'] = df['abstract'].apply(lambda x: get_llm_values(x, keywords_message))
# end_time = time.time()

# # Calculate elapsed time in minutes
# runtime_minutes = (end_time - start_time) / 60
# print(f"Run time for get keywords: {runtime_minutes:.0f} minutes")